In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

#Need to use the latest Tensorflow version - can find it at https://www.tensorflow.org/install/
#!pip uninstall tensorflow
#!pip install tensorflow==2.6.0
import tensorflow as tf
##was 2.5.0, now 2.6.0
import numpy as np
import os
import datetime

In [ ]:
print("TensorFlow version: ", tf.__version__)

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import os
path = os.getcwd()
print(path)

In [ ]:
#If using a directory with multiple files
import glob
import codecs
books = sorted(glob.glob(path + "/../input/datatest/cbtest_V_train.txt"))
print("Found {} books".format(len(books)))

text = ""
for filename in books:
    with codecs.open(filename, 'r', 'utf-8') as books:
        text += books.read()

print("Text is {} characters long".format(len(text)))

In [ ]:
#If using a single file
text = open(path + '/../input/datatest/cbtest_V_train.txt', 'rb').read().decode(encoding='utf-8')
print("Text is {} characters long".format(len(text)))

In [ ]:
words = [w for w in text.split(' ') if w.strip() != '' or w == '\n']
print("Text is {} words long".format(len(words)))

In [ ]:
print(text[:100])

In [ ]:
#Map unique characters to indices
vocab = sorted(set(text))
print ('There are {} unique characters'.format(len(vocab)))
char2int = {c:i for i, c in enumerate(vocab)}
int2char = np.array(vocab)
print('Vector:\n')
for char,_ in zip(char2int, range(len(vocab))):
   print(' {:4s}: {:3d},'.format(repr(char), char2int[char]))

In [ ]:
text_as_int = np.array([char2int[ch] for ch in text], dtype=np.int32)
print ('{}\n mapped to integers:\n {}'.format(repr(text[:100]), text_as_int[:100]))

In [ ]:
#tr_text = text_as_int[:14059004] #text separated for training, divisible by the batch size (64)
#val_text = text_as_int[14059004:] #text separated for validation

tr_text = text_as_int[:25059004]
#val_text = text_as_int[25059004:35059004]
val_text = text_as_int[29059004:33059004]

In [ ]:
print(text_as_int.shape, tr_text.shape, val_text.shape)

In [ ]:
# Populate the library of tunables - I like keeping them centralized in case I need to change things around:
batch_size = 128
buffer_size = 10000
embedding_dim = 256
epochs = 25
seq_length = 200
examples_per_epoch = len(text)//seq_length
#lr = 0.001 #will use default for Adam optimizer
#rnn_units = 1024
rnn_units = 1024
vocab_size = len(vocab)

In [ ]:
tr_char_dataset = tf.data.Dataset.from_tensor_slices(tr_text)
val_char_dataset = tf.data.Dataset.from_tensor_slices(val_text)
print(tr_char_dataset, val_char_dataset)
tr_sequences = tr_char_dataset.batch(seq_length+1, drop_remainder=True)
val_sequences = val_char_dataset.batch(seq_length+1, drop_remainder=True)
print(tr_sequences, val_sequences)

for item in tr_sequences.take(1):
    print(repr(''.join(int2char[item.numpy()])))
    print(item)
for item in val_sequences.take(1):
    print(repr(''.join(int2char[item.numpy()])))
    print(item)

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

tr_dataset = tr_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
val_dataset = val_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
print(tr_dataset, val_dataset)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.2), 
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.2), 
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=batch_size)

In [ ]:
for input_example_batch, target_example_batch in tr_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "respectively: batch_size, sequence_length, vocab_size")

In [ ]:
model.summary()

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
print("Input: \n", repr("".join(int2char[input_example_batch[0]])))
print()
print("Predictions: \n", repr("".join(int2char[sampled_indices ])))

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
def accuracy(labels, logits):
    return tf.keras.metrics.sparse_categorical_accuracy(labels, logits)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
example_batch_acc  = accuracy(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Loss:      ", example_batch_loss.numpy().mean())
print("Accuracy:      ", example_batch_acc.numpy().mean())

In [ ]:
#optimizer = tf.keras.optimizers.Adam() 
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) 
model.compile(optimizer=optimizer, loss=loss)

In [ ]:
patience = 3
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)

In [ ]:

checkpoint_dir = './checkpoints'+ datetime.datetime.now().strftime("_%Y.%m.%d-%H:%M:%S")
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:

history = model.fit(tr_dataset, epochs=epochs, callbacks=[checkpoint_callback, early_stop] , validation_data=val_dataset)
print ("Training stopped as there was no improvement after {} epochs".format(patience))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,9))
plt.plot(history.history['loss'], 'g')
plt.plot(history.history['val_loss'], 'rx') #use if have val data
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper right')
plt.legend(['Train', 'Validation'], loc='upper right') #use if have val date
plt.show()

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
#model.load_weights('./checkpoints_2019.04.29-00:31:15/ckpt_17')  #if the latest checkpoint is not your preferred
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))  #if the latest checkpoint is what you want
model.build(tf.TensorShape([1, None]))
model.summary()

In [ ]:
def generate_text(model, start_string):
    
    print('Generating with seed: "' + start_string + '"')
  
    num_generate = 10000

    input_eval = [char2int[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    #temperature = 1.0
    temperature = 0.5
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(int2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
model.save(r'./fb.h5')
print(generate_text(model, start_string="fighting for life"))

In [ ]:
with open('./sampleTF2.txt', 'w') as f:
    sampleTF2 = generate_text(model, start_string="joy of gods")
    f.write(sampleTF2)

In [ ]:
#model.save('0.0.h5')

In [ ]:
#print(generate_text(model, start_string="<sos> death"))

In [ ]:
#saves as savedmodel
#tf.saved_model.save(model, r'/content/drive/MyDrive/Models')

In [ ]:
#model2 = tf.saved_model.load(r'/content/drive/MyDrive/Models')